In [ ]:
! git clone https://github.com/abudesai/timeVAE.git

Cloning into 'timeVAE'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 241 (delta 51), reused 61 (delta 32), pack-reused 139
Receiving objects: 100% (241/241), 15.54 MiB | 44.46 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [ ]:
! pip install -r /content/timeVAE/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 100.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 113.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 114.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 112.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 41.4 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.2
    Uninstalling pyparsing-

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


#1. Запуск TimeVAE

In [ ]:
import numpy as np
import pandas as pd
import os
import numpy as np, pandas as pd
import time
import joblib
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
import sys
sys.path.append('/content/timeVAE/')
sys.path.append('/content/timeVAE/src')

In [ ]:
from data_utils import (
    load_yaml_file,
    load_data,
    split_data,
    scale_data,
    inverse_transform_data,
    save_scaler,
    save_data,
)

In [ ]:
import paths as paths
from vae.vae_utils import (
    instantiate_vae_model,
    train_vae,
    save_vae_model,
    get_posterior_samples,
    get_prior_samples,
    load_vae_model,
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from visualize import plot_samples, plot_latent_space_samples, visualize_and_save_tsne

In [ ]:
def rolling(data, seq_len):
  temp_data = []
  for i in range(0, len(data) - seq_len):
    _x = data[i:i + seq_len]
    temp_data.append(_x)
  return np.array(temp_data)

#Генерация данных

In [ ]:
import plotly.graph_objects as go

In [ ]:
share = pd.read_csv('/content/drive/MyDrive/kursach/dataset.csv', index_col=0, header=[0, 1])\
      .sort_index(axis=1)['AAPL'].fillna(method='ffill')

In [ ]:
from plotly.subplots import make_subplots
import numpy as np
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
               vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'),
               row_width=[0.2, 0.7])


# Plot OHLC on 1st row
fig.add_trace(go.Candlestick(x=share.index, open=share["open"], high=share["high"],
                low=share["low"], close=share["close"], name="OHLC"),
                row=1, col=1
)

# Bar trace for volumes on 2nd row without legend
fig.add_trace(go.Bar(x=share.index, y=np.log(share['volume']), showlegend=False), row=2, col=1)
fig.update_layout(
        title='AAPL',
        autosize=False,
        width=1000,
        height=500,
    )
# Do not show OHLC's rangeslider plot
fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
def make_gen_data(stock, vae_trend_type):
  ori_data = pd.read_csv('/content/drive/MyDrive/kursach/dataset.csv', index_col=0, header=[0, 1])\
      .sort_index(axis=1)[stock].fillna(method='ffill')

  original_data = rolling(ori_data, 30)
  #так норм делать??
  scaled_train_data, scaled_train_data, scaler = scale_data(original_data, original_data)
  joblib.dump(scaled_train_data, f'/content/drive/MyDrive/kursach/timeVAE/data_vae_{stock}')

  vae_type = "timeVAE"

  hyperparameters = {'latent_dim': 30,
  'hidden_layer_sizes': [50, 100, 200],
  'reconstruction_wt': 3.0,
  'batch_size': 128,
  'use_residual_conn': True,
  'trend_poly': 0,
  'custom_seas': None}

  if vae_trend_type == 'trend':
    hyperparameters['trend_poly'] = 2

  _, sequence_length, feature_dim = scaled_train_data.shape
  vae_model = instantiate_vae_model(
          vae_type=vae_type,
          sequence_length=sequence_length,
          feature_dim=feature_dim,
          **hyperparameters,
      )

  train_vae(
          vae=vae_model,
          train_data=scaled_train_data,
          max_epochs=200,
          verbose=0,
      )
  #мб эпох поменьше

  x_decoded = get_posterior_samples(vae_model, scaled_train_data)
  prior_samples = get_prior_samples(vae_model, num_samples=scaled_train_data.shape[0])

  joblib.dump(x_decoded, f'/content/drive/MyDrive/kursach/timeVAE/decoded_{vae_trend_type}_{stock}')
  joblib.dump(prior_samples, f'/content/drive/MyDrive/kursach/timeVAE/prior_samples_{vae_trend_type}_{stock}')

In [ ]:
make_gen_data('NVDA', 'base')

In [ ]:
make_gen_data('NVDA', 'trend')

In [ ]:
make_gen_data('AAPL', 'base')

In [ ]:
make_gen_data('AAPL', 'trend')

In [ ]:
make_gen_data('ORCL', 'base')

In [ ]:
make_gen_data('ORCL', 'trend')